In [44]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer,  WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import re

In [45]:


import pandas as pd
df = pd.read_csv('train.csv')


df

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\r\n\r\nThis is a letter to...",3
...,...,...,...
17302,ffd378d,"the story "" The Challenge of Exploing Venus "" ...",2
17303,ffddf1f,Technology has changed a lot of ways that we l...,4
17304,fff016d,If you don't like sitting around all day than ...,2
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   17307 non-null  object
 1   full_text  17307 non-null  object
 2   score      17307 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 405.8+ KB


In [47]:
df.describe()

,score
count,17307.000000
mean,2.948402
std,1.044899
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,6.000000


In [48]:
X = df["full_text"]
X

0        Many people have car where they live. The thin...
1        I am a scientist at NASA that is discussing th...
2        People always wish they had the same technolog...
3        We all heard about Venus, the planet without a...
4        Dear, State Senator\r\n\r\nThis is a letter to...
                               ...                        
17302    the story " The Challenge of Exploing Venus " ...
17303    Technology has changed a lot of ways that we l...
17304    If you don't like sitting around all day than ...
17305    In "The Challenge of Exporing Venus," the auth...
17306    Venus is worthy place to study but dangerous. ...
Name: full_text, Length: 17307, dtype: object

In [49]:
y = df["score"]
y

0        3
1        3
2        4
3        4
4        3
        ..
17302    2
17303    4
17304    2
17305    1
17306    2
Name: score, Length: 17307, dtype: int64

In [50]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize the PorterStemmer
stemmer =  SnowballStemmer("english")

def preprocess_text(text):
    
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Apply stemming to each word
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    
    # Join the stemmed words back into a sentence
    stemmed_text = " ".join(stemmed_words)
    
    return stemmed_text

X = X.apply(preprocess_text)
X


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keske\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\keske\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0        mani peopl car live . thing n't know use car a...
1        scientist nasa discuss `` face '' mar . explai...
2        peopl alway wish technolog seen movi , best ne...
3        heard venus , planet without almost oxygen ear...
4        dear , state senat letter argu favor keep elec...
                               ...                        
17302    stori `` challeng explo venus `` inform piec d...
17303    technolog chang lot way live today . nowaday p...
17304    n't like sit around day great opportun partici...
17305    `` challeng expor venus , '' author suggest st...
17306    venus worthi place studi danger . reaosn thei ...
Name: full_text, Length: 17307, dtype: object

In [51]:
X.shape

(17307,)

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
#max_df=0.5, analyzer="word", stop_words ="english"

vectors = vectorizer.fit_transform(X).toarray()
vectors.shape


(17307, 46602)

In [53]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

vectors_modified = scaler.fit_transform(vectors)



In [54]:
pca = PCA(n_components=100)

X = pca.fit_transform(vectors_modified)

X.shape

(17307, 100)

In [55]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:

X_train.shape

(13845, 100)

In [57]:
X_train

array([[-0.26275838, -1.4540641 , -0.2481939 , ..., -3.0171993 ,
         0.80998943, -0.7540348 ],
       [-0.05626606,  2.35154488, -0.90711338, ...,  0.32592391,
         0.03017914,  0.20091424],
       [-0.13880829,  1.54326858, -0.63813253, ...,  0.20711265,
        -0.03209257,  0.0226431 ],
       ...,
       [-0.15895871,  0.25210053, -0.49447391, ...,  1.38782859,
        -0.90865938,  0.92902688],
       [-0.21864099,  4.37187256, -1.30210022, ...,  0.20895037,
         1.08912361, -0.09404291],
       [-0.17334852,  3.5747328 , -1.62608241, ...,  3.87512   ,
        -1.56744886, -2.45816979]])

In [58]:
X_test.shape

(3462, 100)

In [59]:
X_test

array([[ 0.12146545, -7.78172349,  0.76622798, ..., -0.59403661,
         1.158326  , -0.19782985],
       [-0.24405799,  5.39064074, -1.22346687, ..., -0.33680177,
        -0.53255555, -0.02288904],
       [-0.17723496,  3.10927293, -0.76172441, ..., -0.39357685,
         0.11560728, -0.29153545],
       ...,
       [-0.14360153,  2.9709629 , -1.3384101 , ...,  0.78428386,
        -0.0646788 ,  0.52663488],
       [-0.04922457,  2.0444523 , -0.98764094, ...,  0.19353318,
        -0.05074598, -0.21771477],
       [-0.08412149,  2.72119331, -0.87431768, ...,  0.10429505,
        -0.18251958, -0.18620802]])

In [60]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# import xgboost as xgb
# from xgb import XGBClassifier

# Initialize models
models = {
    # 'Multinomial Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine': SVC(kernel='linear'),
    'Random Forest Classifier': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier(),
    # 'XGBoost Classifier' : XGBClassifier(),
    'Neural Network': MLPClassifier(max_iter=1000)
}


# Train and evaluate each model
for name, model in models.items():
    print(f"{name} ")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Calculate precision
    precision = precision_score(y_test, y_pred, average='weighted')
    print("Precision:", precision)

    # Calculate recall
    recall = recall_score(y_test, y_pred, average='weighted')
    print("Recall:", recall)

    # Calculate F1-score
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("F1-score:", f1)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("Confusion matrix:")
    print(conf_matrix)

Logistic Regression 
Accuracy: 0.5569035239745812
Precision: 0.5469551241789973
Recall: 0.5569035239745812
F1-score: 0.5424615150547746
Confusion matrix:
[[ 35 169  48   7   0   1]
 [ 24 642 270  27   0   2]
 [ 20 298 812 131   3   1]
 [  2  16 313 379  34   6]
 [  1   0   7 117  49   9]
 [  0   0   0   6  22  11]]
r2_score = 0.45492485736920485
mse = 0.6013864818024264
Support Vector Machine 
Accuracy: 0.5618139803581744
Precision: 0.5433440505979913
Recall: 0.5618139803581744
F1-score: 0.5414281736535433
Confusion matrix:
[[ 13 193  43  10   0   1]
 [ 15 668 251  29   2   0]
 [ 14 309 787 153   2   0]
 [  1  16 288 416  25   4]
 [  0   1   6 119  48   9]
 [  0   0   0   7  19  13]]
r2_score = 0.4722038964727747
mse = 0.5823223570190641
Random Forest Classifier 
Accuracy: 0.5814558058925476
Precision: 0.5807055025215634
Recall: 0.5814558058925476
F1-score: 0.5672865774048815
Confusion matrix:
[[ 31 155  69   5   0   0]
 [ 14 590 329  30   2   0]
 [  8 210 856 189   2   0]
 [  1   8 24

In [61]:
df = pd.DataFrame({'y_true': y_pred, 'y_pred': y_test.to_numpy()})

df 

,y_true,y_pred
0,4,3
1,4,3
2,2,3
3,2,3
4,4,4
...,...,...
3457,2,1
3458,1,2
3459,2,2
3460,2,2


In [ ]:
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)


# Initialize models
models = {
    # 'Multinomial Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine': SVC(kernel='linear'),
    'Random Forest Classifier': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier(),
    # 'XGBoost Classifier' : XGBClassifier(),
    'Neural Network': MLPClassifier(max_iter=1000)
}


# Train and evaluate each model
for name, model in models.items():
    accuracy_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    precision_scores = cross_val_score(model, X, y, cv=kf, scoring='precision_weighted')
    recall_scores = cross_val_score(model, X, y, cv=kf, scoring='recall_weighted')
    f1_scores = cross_val_score(model, X, y, cv=kf, scoring='f1_weighted')

    # Print average scores
    print("Average Accuracy:", np.mean(accuracy_scores))
    print("Average Precision:", np.mean(precision_scores))
    print("Average Recall:", np.mean(recall_scores))
    print("Average F1-score:", np.mean(f1_scores))